# Homework 1

### DUE: 02/09/2017 before class at 10:30am

This homework is for practicing Python’s <i>generators</i> and <i>streaming</i>. We’re going to use the Citibike dataset posted on NYU Classes. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW1\_<b>NetId</b>.ipynb.



## Task 1 (4 points)

Your task is to <b>compute the median age</b> of the Citibike’s <b>subscribed</b> customers, aka. usertype is "Subscriber". You are required to read data line by line and are not allowed to store the entire data set in memory. Indeed, you should not have any containers (e.g. list, dictionary, DataFrame, etc.) with more than a handful, e.g. strictly < 100, of elements in memory either as a local or a global variable. You can use the citibike.csv data file that we have on NYU Classes for testing, but we will evaluate your code on a much larger input to ensure it’s streaming capability.

The code block below is taken mostly from our lab and would stream data from the citibike.csv for on-demand processing. The data file should be stored on the same folder with this notebook. You will need to replace the portion inside the for loop with your own code.

In [1]:
import csv

# generator that yields rows of a text file, in a stream
def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row
            
### NOTE: You can initialize any global variables below,
### but they should hold less than 100 elements.

#<IAN'S CODE HERE>
this_year = 2017
age_counts = {}
running_total = 0

for row in csvRows('citibike.csv'):
    ### NOTE: perform your streaming computation here. 'row' is a
    ### tuple of values for the current record of the input file.
    ### You should replace 'pass' below with your code.
    
    #<IAN'S CODE HERE>
    # check if user is a subscriber
    if row['usertype'] == 'Subscriber':
        # compute individual's age
        age = this_year - int(row['birth_year'])
        # add to counting dict
        age_counts[age] = age_counts.get(age, 0) + 1
    else:
        pass # don't count these users
    
        
### NOTE: You can do further processing to get the median age value
###
medianAge = 0

#<IAN'S CODE HERE>
# summing an iterator to get total count of riders, without using much memory
overall_total = sum(age_counts.itervalues())
for k in sorted(age_counts.keys()):
    # compute a running total of rider count, in order from youngest to oldest
    running_total += age_counts[k]
    # when the running total exceeds the midway mark of the true total,
    # we know we're sitting at the median key
    if running_total*2 >= overall_total:
        medianAge = k
        break
    
###
print 'median age:', medianAge

median age: 40


## Task 2 (6 points)

Your task is to write a generator to extract the first ride of the day from a Citibike data stream. The data stream is sorted based on starting times (similar to the <b>citibike.csv</b> file uploaded on NYU Classes). The first ride of the day is interpreted as the ride with the earliest starting time of a day. For the sample data, which is a week worth of citibike records, your generator should only generate 7 items (one for each day).

You are given a template with the sample generator <b>firstRide</b>. The generator currently takes in <b>csv.DictReader</b> generator and output its first element. Please adjust this generator to output the first ride of the day for the entire stream as specified above. The output of the generator must be in the same format as csv.DictReader. You can think of this generator as a filter only passing certain records out. \

In [39]:
import csv
import datetime

### NOTE: You need to change the body of the generator firstRide
### in order to output trip record that appeared first in each day
### using the same dict format as csv.DictReader.

def firstRide(reader):
    
    #<IAN'S CODE HERE>
    date_marker = None
    for row in reader:
        # remove the timezone characters from end of date string, before parsing
        ride_str = row['starttime'][:-3]
        ride_date = datetime.datetime.strptime(ride_str, "%Y-%m-%d %H:%M:%S").date()
        # the first ride of the day is flagged when the ride date doesn't match current marker
        if ride_date != date_marker:
            # reset the date_marker
            date_marker = ride_date
            # yield the "filtered" row
            yield row
        else:
            pass # not a first ride, skip this row

### NOTE: You SHOULD NOT modify the code below. If you
### write your firstRide generator above correctly, the
### code below will output the correct information

with open('citibike.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    for row in firstRide(reader):
        print ','.join(map(row.get, reader.fieldnames))


1,,801,2015-02-01 00:00:00+00,2015-02-01 00:14:00+00,521,8 Ave & W 31 St,40.75044999,-73.99481051,423,W 54 St & 9 Ave,40.76584941,-73.98690506,17131,Subscriber,1978,2
6442,,199,2015-02-02 00:02:00+00,2015-02-02 00:05:00+00,442,W 27 St & 7 Ave,40.746647,-73.993915,489,10 Ave & W 28 St,40.75066386,-74.00176802,20684,Subscriber,1992,1
7901,,704,2015-02-03 00:00:00+00,2015-02-03 00:12:00+00,387,Centre St & Chambers St,40.71273266,-74.0046073,2008,Little West St & 1 Pl,40.70569254,-74.01677685,20328,Subscriber,1982,1
12655,,146,2015-02-04 00:00:00+00,2015-02-04 00:02:00+00,237,E 11 St & 2 Ave,40.73047309,-73.98672378,438,St Marks Pl & 1 Ave,40.72779126,-73.98564945,15253,Subscriber,1969,1
21628,,1034,2015-02-05 00:00:00+00,2015-02-05 00:17:00+00,497,E 17 St & Broadway,40.73704984,-73.99009296,461,E 20 St & 2 Ave,40.73587678,-73.98205027,20290,Subscriber,1971,1
30836,,212,2015-02-06 00:01:00+00,2015-02-06 00:05:00+00,491,E 24 St & Park Ave S,40.74096374,-73.98602213,472,E 32 St & Park Ave,40